In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="tqdm")

import logging
logging.getLogger('rpy2').setLevel(logging.WARNING)

%load_ext autoreload
%autoreload 2
# from ipython import autoreload
import numpy as np
import itertools
import yaml
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import causallearn
from causallearn.graph.Edge import Edge
from causallearn.graph.Endpoint import Endpoint
from causallearn.search.ConstraintBased.FCI import fci
import random

from clustercausal.algorithms.ClusterPC import ClusterPC
from clustercausal.algorithms.ClusterFCI import ClusterFCI
from clustercausal.clusterdag.ClusterDAG import ClusterDAG
from clustercausal.experiments.Evaluator import Evaluator
# from clustercausal.experiments.ExperimentRunner import ExperimentRunner
from clustercausal.experiments.ExperimentRunnervsFCITiers import (
    ExperimentRunner,
)
from clustercausal.utils.Utils import *
from clustercausal.experiments.Utils import load_experiment_folder
from clustercausal.utils.Utils import *
from clustercausal.experiments.Simulator import Simulator
from clustercausal.algorithms.FCITiers import fci_tiers

c:\Users\ga87val\00_PhD\Paper Development\ClusterDAGs_causal_discovery\clustercausal\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


In [2]:
data = load_experiment_folder('clustercausal\experiments\_results')

In [3]:
arrow_metrics = ['base_arrow_precision', 'cluster_arrow_precision', 'fcitiers_arrow_precision',\
                'base_arrow_recall', 'cluster_arrow_recall', 'fcitiers_arrow_recall',\
                'base_arrow_f1_score', 'cluster_arrow_f1_score', 'fcitiers_arrow_f1_score']
prec_recall_metrics_with_fcitiers = ['base_adj_precision', 'cluster_adj_precision', 'fcitiers_adj_precision',\
              'base_adj_recall', 'cluster_adj_recall', 'fcitiers_adj_recall', \
                'base_adj_f1_score', 'cluster_adj_f1_score', 'fcitiers_adj_f1_score']
extended_metrics = prec_recall_metrics_with_fcitiers + ['base_base_shd', 'cluster_cluster_shd', 'fcitiers_fcitiers_shd', 'Base indep tests', 'Cluster indep tests']
all_metrics = extended_metrics + arrow_metrics
data[all_metrics]

,base_adj_precision,cluster_adj_precision,fcitiers_adj_precision,base_adj_recall,cluster_adj_recall,fcitiers_adj_recall,base_adj_f1_score,cluster_adj_f1_score,fcitiers_adj_f1_score,base_base_shd,...,Cluster indep tests,base_arrow_precision,cluster_arrow_precision,fcitiers_arrow_precision,base_arrow_recall,cluster_arrow_recall,fcitiers_arrow_recall,base_arrow_f1_score,cluster_arrow_f1_score,fcitiers_arrow_f1_score
0,0.952381,0.875000,0.900000,0.606061,0.636364,0.545455,0.740741,0.736842,0.679245,18.0,...,1620.0,0.705882,0.818182,0.833333,0.363636,0.545455,0.454545,0.480000,0.654545,0.588235
1,0.944444,0.944444,0.944444,0.708333,0.708333,0.708333,0.809524,0.809524,0.809524,11.0,...,1621.0,0.750000,0.812500,0.700000,0.500000,0.541667,0.583333,0.600000,0.650000,0.636364
2,1.000000,0.909091,0.952381,0.833333,0.833333,0.833333,0.909091,0.869565,0.888889,13.0,...,1492.0,0.400000,0.681818,0.740741,0.250000,0.625000,0.833333,0.307692,0.652174,0.784314
3,0.900000,0.869565,0.850000,0.562500,0.625000,0.531250,0.692308,0.727273,0.653846,20.0,...,1696.0,0.625000,0.666667,0.714286,0.312500,0.437500,0.468750,0.416667,0.528302,0.566038
4,1.000000,0.950000,1.000000,0.863636,0.863636,0.863636,0.926829,0.904762,0.926829,7.0,...,814.0,0.750000,0.941176,1.000000,0.545455,0.727273,0.727273,0.631579,0.820513,0.842105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.822222,0.685185,0.804348,0.948718,0.948718,0.948718,0.880952,0.795699,0.870588,12.0,...,2659.0,0.655172,0.725490,0.787234,0.487179,0.948718,0.948718,0.558824,0.822222,0.860465
156,0.509434,0.444444,0.509434,0.900000,0.933333,0.900000,0.650602,0.602151,0.650602,36.0,...,2418.0,0.319149,0.444444,0.435484,0.500000,0.933333,0.900000,0.389610,0.602151,0.586957
157,0.734694,0.580645,0.705882,1.000000,1.000000,1.000000,0.847059,0.734694,0.827586,18.0,...,3026.0,0.552632,0.590164,0.666667,0.583333,1.000000,1.000000,0.567568,0.742268,0.800000
158,0.625000,0.510204,0.555556,0.925926,0.925926,0.925926,0.746269,0.657895,0.694444,22.0,...,1688.0,0.370370,0.543478,0.543478,0.370370,0.925926,0.925926,0.370370,0.684932,0.684932


In [4]:
prec_recall_metrics_with_pruned = ['base_adj_precision', 'pruned_base_adj_precision', 'fcitiers_adj_precision', 'cluster_adj_precision', \
           'base_adj_recall', 'pruned_base_adj_recall', 'fcitiers_adj_recall', 'cluster_adj_recall', \
           'base_adj_f1_score', 'pruned_base_adj_f1_score',  'fcitiers_adj_f1_score', 'cluster_adj_f1_score']
data[prec_recall_metrics_with_pruned]
f1_score_metrics = ['base_adj_f1_score', 'cluster_adj_f1_score', 'fcitiers_adj_f1_score', 'pruned_base_adj_f1_score']
data[f1_score_metrics]

,base_adj_f1_score,cluster_adj_f1_score,fcitiers_adj_f1_score,pruned_base_adj_f1_score
0,0.740741,0.736842,0.679245,0.740741
1,0.809524,0.809524,0.809524,0.809524
2,0.909091,0.869565,0.888889,0.909091
3,0.692308,0.727273,0.653846,0.692308
4,0.926829,0.904762,0.926829,0.926829
...,...,...,...,...
155,0.880952,0.795699,0.870588,0.880952
156,0.650602,0.602151,0.650602,0.650602
157,0.847059,0.734694,0.827586,0.847059
158,0.746269,0.657895,0.694444,0.746269


load from a graph

In [5]:
#clustercausal\experiments\_results\ClusterFCI_2024-12-20 11-58-31.118336\erdos_renyi_15_nodes_20_edges_7_clusters_gauss_run8_11-59-40-170
path = "clustercausal\experiments\_results\\test\erdos_renyi_15_nodes_30_edges_7_clusters_gauss_run8_12-04-26-628"
path = "clustercausal\experiments\_results\\test\erdos_renyi_15_nodes_30_edges_7_clusters_gauss_run9_12-04-27-096"
# path = "clustercausal\experiments\_results\\test\erdos_renyi_15_nodes_30_edges_7_clusters_gauss_run10_12-04-27-676"
base_cg, cluster_cg, tiers_cg, cdag = load_experiment_graphs(path)
cdag.cluster_mapping
cdag.cluster_graph.draw_pydot_graph()
base_cg.draw_pydot_graph()
tiers_cg.draw_pydot_graph()
cluster_cg.draw_pydot_graph()
with open(f"{path}\\results.yaml") as file:
    results = yaml.load(file, Loader=yaml.FullLoader)

UnboundLocalError: cannot access local variable 'base_est_graph' where it is not associated with a value

In [ ]:
pd.DataFrame(results)

,base_evaluation_results,cluster_evaluation_results,fcitiers_evaluation_results,pruned_base_evaluation_results,settings
adj_f1_score,0.250000,0.333333,0.269231,0.250000,NaN
adj_false_negative,31.000000,28.000000,30.000000,31.000000,NaN
adj_false_positive,5.000000,8.000000,8.000000,5.000000,NaN
adj_precision,0.545455,0.529412,0.466667,0.545455,NaN
adj_recall,0.162162,0.243243,0.189189,0.162162,NaN
adj_true_negative,63.000000,60.000000,60.000000,63.000000,NaN
adj_true_positive,6.000000,9.000000,7.000000,6.000000,NaN
arrow_f1_score,0.066667,0.315789,0.222222,0.066667,NaN
arrow_f1_score_ce,0.333333,0.857143,0.875000,0.333333,NaN
arrow_false_negative,46.000000,36.000000,41.000000,46.000000,NaN
